# STAT207 Final Project - INSERT TITLE HERE


Afif Shomali, Jasmine Yan, Wentian Wang, Wahid Nasir

**Delete this code block before submitting**

You should delete and add cells as you see fit.

Don't forget to put written text in **markdown** cells. To change a cell from a <u>code block</u> to a <u>markdown cell</u>:
* Click on the cell you'd like to modify.
* Go to 'Cell' at the top of the page.
* Go to 'Cell Type' in the drop down menu and click on either 'Code' or 'Markdown'.


In [26]:
#Imports here
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso, Ridge
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

## 1. Introduction

### Primary Reasearch Goal

In this project our primary research goal will be to build a predictive model that will perform the best when predicting "Highly Rated" video games for new video games given some combination of the other remaining explanatory variables. For our purposes, a video game is considered "Highly Rated" when it has ``Metric.Review.Score`` of greater than 80. We will create our own response variable with 2 levels based on this review score and constraint to categorize games.


### Secondary Research Goal

Our secondary research goal is to have a model that has reliable interpretative insights about the nature of the relationship between the variables in the dataset. We would also like to thoroughly describe the nature of the variables and well as the relationships between the variables that we would like to use in our model.



## 2. Dataset Discussion

### TODO 

Add some writing/description as to what the dataset is, etc. Can use text from project specification as a guide see the Dataset Options section.

In [27]:
df = pd.read_csv("video_games.csv")
df.head()

,Title,Features.Handheld?,Features.Max Players,Features.Multiplatform?,Features.Online?,Metadata.Genres,Metadata.Licensed?,Metadata.Publishers,Metadata.Sequel?,Metrics.Review Score,...,Length.Main + Extras.Average,Length.Main + Extras.Leisure,Length.Main + Extras.Median,Length.Main + Extras.Polled,Length.Main + Extras.Rushed,Length.Main Story.Average,Length.Main Story.Leisure,Length.Main Story.Median,Length.Main Story.Polled,Length.Main Story.Rushed
0,Super Mario 64 DS,True,1,True,True,Action,True,Nintendo,True,85,...,24.916667,29.966667,25.000000,16,18.333333,14.333333,18.316667,14.500000,21,9.700000
1,Lumines: Puzzle Fusion,True,1,True,True,Strategy,True,Ubisoft,True,89,...,9.750000,9.866667,9.750000,2,9.616667,10.333333,11.083333,10.000000,3,9.583333
2,WarioWare Touched!,True,2,True,True,"Action,Racing / Driving,Sports",True,Nintendo,True,81,...,3.850000,5.666667,3.333333,11,2.783333,1.916667,2.933333,1.833333,30,1.433333
3,Hot Shots Golf: Open Tee,True,1,True,True,Sports,True,Sony,True,81,...,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000,0,0.000000
4,Spider-Man 2,True,1,True,True,Action,True,Activision,True,61,...,12.766667,17.316667,12.500000,12,10.483333,8.350000,11.083333,8.000000,23,5.333333


In [28]:
df.shape[0]

1212

## 3. Dataset Cleaning

During our dataset cleaning we will add the categorical variable that we will be trying to predict, handle any Null or missing values. We will also get rid of some of the unecessary columns that we won't be using for a model.

First we will handle any Null or missing values appropriately. The only column with NaN values is the publisher column, for simplicity we will replace those values with "Unknown". 

In [29]:
print(df.columns[df.isna().any()])


df["Metadata.Publishers"] = df["Metadata.Publishers"].fillna("Unknown")

Index(['Metadata.Publishers'], dtype='object')


We will remove any columns that we won't be using or won't be helpful. For examples some columns have the exact same value for all the datapoints so they wouldn't be beneficial in helping us predict highly rated games. This also will let us build a simpler & more interpretable model which helps us with our secondary research goal. 

In [30]:
df = df[["Title", "Features.Max Players", "Metadata.Genres", 
        "Metadata.Publishers", "Metrics.Review Score", "Metrics.Sales",
        "Metrics.Used Price", "Release.Console", "Release.Rating",
        "Release.Year", "Length.All PlayStyles.Average"]]

In [32]:
df.head(5)

,Title,Features.Max Players,Metadata.Genres,Metadata.Publishers,Metrics.Review Score,Metrics.Sales,Metrics.Used Price,Release.Console,Release.Rating,Release.Year,Length.All PlayStyles.Average,Highly.Rated
0,Super Mario 64 DS,1,Action,Nintendo,85,4.69,24.95,Nintendo DS,E,2004,22.716667,1
1,Lumines: Puzzle Fusion,1,Strategy,Ubisoft,89,0.56,14.95,Sony PSP,E,2004,10.100000,1
2,WarioWare Touched!,2,"Action,Racing / Driving,Sports",Nintendo,81,0.54,22.95,Nintendo DS,E,2004,4.566667,1
3,Hot Shots Golf: Open Tee,1,Sports,Sony,81,0.49,12.95,Sony PSP,E,2004,0.000000,1
4,Spider-Man 2,1,Action,Activision,61,0.45,14.95,Nintendo DS,E,2004,13.250000,0


Next we add the ``Highly.Rated`` Response variable that we will be trying to predict, as we see below there is a significant enough amount of rows for both games that are considered highly rated and those games that are not considered highly rated.

In [31]:
df["Highly.Rated"] = df['Metrics.Review Score'].apply(lambda x: 1 if x >= 80 else 0)
df["Highly.Rated"].value_counts()

0    930
1    282
Name: Highly.Rated, dtype: int64

Finally we will double check for any implicit missing values in this dataset in the remaining columns that we plan on using. Based on the datatypes & looking at the unique values, we don't see any implicitly missing values.

In [33]:
df.dtypes

Title                             object
Features.Max Players               int64
Metadata.Genres                   object
Metadata.Publishers               object
Metrics.Review Score               int64
Metrics.Sales                    float64
Metrics.Used Price               float64
Release.Console                   object
Release.Rating                    object
Release.Year                       int64
Length.All PlayStyles.Average    float64
Highly.Rated                       int64
dtype: object

In [37]:
df["Release.Rating"].unique()

array(['E', 'M', 'T'], dtype=object)

In [39]:
df["Release.Console"].unique()

array(['Nintendo DS', 'Sony PSP', 'X360', 'Nintendo Wii', 'PlayStation 3'],
      dtype=object)

## 4. Preliminary Analysis

## 5. Model Data Preprocessing

## 6. Feature Selection with k-Fold Cross-Validation

## 7. Best Model Discussion

## 8. Additional Analysis/Insight

## 9. Conclusion

## References